In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,24999
2,Huelva,Confirmados PDIA 14 días,350
3,Huelva,Tasa PDIA 14 días,"68,20351930159596"
4,Huelva,Confirmados PDIA 7 días,204
5,Huelva,Total Confirmados,25197
6,Huelva,Curados,19022
7,Huelva,Fallecidos,345


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  24999.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6623.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 4359 personas en los últimos 7 días 

Un positivo PCR cada 2569 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,24999.0,204.0,350.0,513170.0,39.752908,68.203519,40.0
Huelva-Costa,15214.0,91.0,177.0,289548.0,31.428295,61.129761,25.0
Condado-Campiña,7063.0,103.0,161.0,156231.0,65.928017,103.052531,15.0
Almonte,905.0,27.0,37.0,24507.0,110.172604,150.977272,9.0
Huelva (capital),6623.0,33.0,56.0,143837.0,22.942636,38.932959,9.0
Isla Cristina,2004.0,10.0,21.0,21393.0,46.744262,98.162950,6.0
Alosno,128.0,6.0,7.0,3933.0,152.555301,177.981185,3.0
Moguer,849.0,14.0,20.0,21867.0,64.023414,91.462020,3.0
Aljaraque,928.0,3.0,3.0,21474.0,13.970383,13.970383,2.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Puebla de Guzmán,108.0,6.0,15.0,3092.0,194.049159,485.122898,0.0
San Bartolomé de la Torre,210.0,7.0,15.0,3761.0,186.120713,398.830098,0.0
Villalba del Alcor,456.0,9.0,11.0,3366.0,267.379679,326.797386,0.0
Corteconcepción,24.0,1.0,1.0,536.0,186.567164,186.567164,0.0
Alosno,128.0,6.0,7.0,3933.0,152.555301,177.981185,3.0
Ayamonte,1232.0,10.0,32.0,21104.0,47.384382,151.630023,1.0
Almonte,905.0,27.0,37.0,24507.0,110.172604,150.977272,9.0
Villarrasa,132.0,3.0,3.0,2211.0,135.685210,135.685210,0.0
Rociana del Condado,320.0,7.0,10.0,7939.0,88.172314,125.960448,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
San Juan del Puerto,367.0,2.0,9.0,9411.0,21.251727,95.632770,0.0,0.222222
Ayamonte,1232.0,10.0,32.0,21104.0,47.384382,151.630023,1.0,0.312500
Puebla de Guzmán,108.0,6.0,15.0,3092.0,194.049159,485.122898,0.0,0.400000
San Bartolomé de la Torre,210.0,7.0,15.0,3761.0,186.120713,398.830098,0.0,0.466667
Isla Cristina,2004.0,10.0,21.0,21393.0,46.744262,98.162950,6.0,0.476190
Huelva-Costa,15214.0,91.0,177.0,289548.0,31.428295,61.129761,25.0,0.514124
Lepe,2008.0,8.0,15.0,27880.0,28.694405,53.802009,2.0,0.533333
Huelva,24999.0,204.0,350.0,513170.0,39.752908,68.203519,40.0,0.582857
Huelva (capital),6623.0,33.0,56.0,143837.0,22.942636,38.932959,9.0,0.589286
